In [1]:
#connect to your drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# print the system info
import torch
import psutil
import platform
import shutil
import subprocess
import sys

def get_cuda_torch_info():
    print("CUDA and PyTorch Versions:")
    if torch.cuda.is_available():
        print(f"  CUDA Version: {torch.version.cuda}")
        print(f"  PyTorch Version: {torch.__version__}")
        print(f"  GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
    else:
        print("  CUDA not available")
        print(f"  PyTorch Version: {torch.__version__}")
    print()

def get_system_info():
    print("System Information:")
    print(f"  System: {platform.system()}")
    print(f"  Node Name: {platform.node()}")
    print(f"  Release: {platform.release()}")
    print(f"  Version: {platform.version()}")
    print(f"  Machine: {platform.machine()}")
    print(f"  Processor: {platform.processor()}")
    print()

def get_cpu_info():
    print("CPU Information:")
    cpu_count = psutil.cpu_count(logical=True)
    print(f"  Logical CPUs: {cpu_count}")
    cpu_freq = psutil.cpu_freq()
    print(f"  CPU Frequency: {cpu_freq.current:.2f} MHz (Max: {cpu_freq.max:.2f} MHz)")
    print()

def get_memory_info():
    print("Memory Information:")
    virtual_mem = psutil.virtual_memory()
    print(f"  Total Memory: {virtual_mem.total / 1e9:.2f} GB")
    print(f"  Available Memory: {virtual_mem.available / 1e9:.2f} GB")
    print(f"  Used Memory: {virtual_mem.used / 1e9:.2f} GB")
    print(f"  Memory Usage: {virtual_mem.percent}%")
    print()

def get_disk_info():
    print("Disk Information:")
    total, used, free = shutil.disk_usage("/")
    print(f"  Total Disk Space: {total / 1e9:.2f} GB")
    print(f"  Used Disk Space: {used / 1e9:.2f} GB")
    print(f"  Free Disk Space: {free / 1e9:.2f} GB")
    print()

def get_gpu_info():
    try:
        import GPUtil
        gpus = GPUtil.getGPUs()
        print("GPU Information:")
        for gpu in gpus:
            print(f"  GPU ID: {gpu.id}")
            print(f"  GPU Name: {gpu.name}")
            print(f"  GPU Load: {gpu.load * 100:.1f}%")
            print(f"  GPU Memory Free: {gpu.memoryFree:.1f} MB")
            print(f"  GPU Memory Used: {gpu.memoryUsed:.1f} MB")
            print(f"  GPU Memory Total: {gpu.memoryTotal:.1f} MB")
            print(f"  GPU Temperature: {gpu.temperature} °C")
            print()
    except ImportError:
        print("GPUtil is not installed. Please install it with `pip install gputil` to get detailed GPU information.")
        print()

def get_python_packages():
    print("Python Packages:")
    installed_packages = subprocess.run(
        [sys.executable, "-m", "pip", "list"],  # Use sys.executable instead of platform.python_executable
        stdout=subprocess.PIPE,
        text=True
    ).stdout
    print(installed_packages)

# Call all functions
get_cuda_torch_info()
get_system_info()
get_cpu_info()
get_memory_info()
get_disk_info()
get_gpu_info()
get_python_packages()


CUDA and PyTorch Versions:
  CUDA Version: 12.1
  PyTorch Version: 2.5.1+cu121
  GPU: NVIDIA A100-SXM4-40GB

System Information:
  System: Linux
  Node Name: 390ec8006aca
  Release: 6.1.85+
  Version: #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
  Machine: x86_64
  Processor: x86_64

CPU Information:
  Logical CPUs: 12
  CPU Frequency: 2200.21 MHz (Max: 0.00 MHz)

Memory Information:
  Total Memory: 89.63 GB
  Available Memory: 86.95 GB
  Used Memory: 1.83 GB
  Memory Usage: 3.0%

Disk Information:
  Total Disk Space: 253.06 GB
  Used Disk Space: 35.00 GB
  Free Disk Space: 218.04 GB

GPUtil is not installed. Please install it with `pip install gputil` to get detailed GPU information.

Python Packages:
Package                            Version
---------------------------------- -------------------
absl-py                            1.4.0
accelerate                         1.1.1
aiohappyeyeballs                   2.4.4
aiohttp                            3.11.9
aiosignal         

In [ ]:
# print privious result
# you can change the number at the end to see the last second one or last third one
import os
import glob
import pickle
from datetime import datetime
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow

def process_latest_folder(n):
    results_folder = "results"
    if not os.path.exists(results_folder):
        print(f"'{results_folder}' does not exist.")
        return

    subfolders = [f for f in os.listdir(results_folder) if os.path.isdir(os.path.join(results_folder, f))]
    subfolders = sorted(
        subfolders,
        key=lambda f: os.path.getmtime(os.path.join(results_folder, f)),
        reverse=True
    )

    if n > len(subfolders):
        print(f"Folder {n} does not exist. Only {len(subfolders)} folders found.")
        return

    target_folder = os.path.join(results_folder, subfolders[n - 1])
    print(f"Target folder: {os.path.abspath(target_folder)}")

    image_files = glob.glob(os.path.join(target_folder, "*.png")) + glob.glob(os.path.join(target_folder, "*.jpg"))
    if image_files:
        print("Image files:")
        for img_file in image_files:
            print(f"  {os.path.abspath(img_file)}")
            image = cv2.imread(img_file)
            cv2_imshow(image)
    else:
        print("No image files found.")

    pkl_files = glob.glob(os.path.join(target_folder, "*.pkl"))
    if len(pkl_files) != 1:
        print(f"{len(pkl_files)} .pkl files found. Please check.")
        return

    pkl_file = pkl_files[0]
    print(f"\nLoaded .pkl file: {os.path.abspath(pkl_file)}")
    with open(pkl_file, "rb") as f:
        loaded_results = pickle.load(f)

    print("Loaded Results:")

    grouped_results = {}
    for key, value in loaded_results.items():
        group_key = key[1]
        if group_key not in grouped_results:
            grouped_results[group_key] = []
        grouped_results[group_key].append((key, value))

    for group_key, items in grouped_results.items():
        print(f"\nGroup '{group_key}':")
        for key, value in items:
            print(f"  {key}: {value}")


process_latest_folder(1)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# will install all the dependencies
!bash /content/drive/MyDrive/Colab\ Notebooks/attention_benchmark/install_dependencies.sh


import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/attention_benchmark')



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61396 sha256=f598a6b77fc9cef74cafa59c0bcdc11c495528409e0af35a3f90ce5142d8cbd0
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31528 sha256=2247114ed7a08ca07ab981d6c13a73e990563db48baaf6545142a8847ee8d6a3
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# will run the code from main.py
!find . -type d -name "__pycache__" -exec rm -rf {} +

!python main.py

2024-12-07 00:37:47.949824: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-07 00:37:47.966441: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-07 00:37:47.987563: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-07 00:37:47.993912: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-07 00:37:48.008993: I tensorflow/core/platform/cpu_feature_guar